# Shailja Somani, John Vincent Deniega, Muris Saab
# ADS 508 - Team 2
# Spring 2024

## Initialize SageMaker variables: Direct copy from

Coyne, S. (2024, February 27). Update 01_Copy_TSV_To_S3.ipynb [Jupyter Notebook]. GitHub. https://github.com/MADS508/labs/blob/main/04_ingest/01_Copy_TSV_To_S3.ipynb

In [2]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


## Import PyAthena

Coyne, S. (2022, October 9). 02_Create_Athena_Database.ipynb [Jupyter Notebook]. GitHub. https://github.com/MADS508/labs/commits/main/04_ingest/02_Create_Athena_Database.ipynb

In [3]:
!pip install --disable-pip-version-check -q PyAthena==2.1.0
from pyathena import connect

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


## Create Athena Database

Coyne, S. (2022, October 9). 02_Create_Athena_Database.ipynb [Jupyter Notebook]. GitHub. https://github.com/MADS508/labs/commits/main/04_ingest/02_Create_Athena_Database.ipynb

In [4]:
database_name = "covid"
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)
pd.read_sql(statement, conn)

CREATE DATABASE IF NOT EXISTS covid


""


## Verify Athena Database

Coyne, S. (2022, October 9). 02_Create_Athena_Database.ipynb [Jupyter Notebook]. GitHub. https://github.com/MADS508/labs/commits/main/04_ingest/02_Create_Athena_Database.ipynb

In [5]:
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

,database_name
0,covid
1,default
2,dsoaws


## Set S3 bucket paths into variables

In [6]:
MDCovid_path = "s3://ads508-team2-spring24/01_MD_COVID-19_-_Cases_by_ZIP_Code.csv"
SDOH_path = "s3://ads508-team2-spring24/02_SDOH_2020_ZIPCODE_1_0.xlsx"
CDC_path = "s3://ads508-team2-spring24/03_PLACES__Local_Data_for_Better_Health__ZCTA_Data_2023_release.csv"
MDCensus_path = "s3://ads508-team2-spring24/04_Maryland_Census_Data_-_ZIP_Code_Tabulation_Areas_(ZCTAs).csv"
MDCrosswalk_path = "s3://ads508-team2-spring24/05_ZIPCodetoZCTACrosswalk2022UDS.xlsx"

## Ingest five data sources into Pandas DataFrame

In [7]:
MDCovid = pd.read_csv("dataset_csvs/01_MD_COVID-19_-_Cases_by_ZIP_Code.csv")
SDOH = pd.read_excel("dataset_csvs/02_SDOH_2020_ZIPCODE_1_0.xlsx") # Troubleshoot why selecting a sheet slows this down
CDC = pd.read_csv("dataset_csvs/03_PLACES__Local_Data_for_Better_Health__ZCTA_Data_2023_release.csv")
MDCensus = pd.read_csv("dataset_csvs/04_Maryland_Census_Data_-_ZIP_Code_Tabulation_Areas_(ZCTAs).csv")
MDCrosswalk = pd.read_excel("dataset_csvs/05_ZIPCodetoZCTACrosswalk2022UDS.xlsx")

# Use to verify data ingest, but commented out to save on computing costs on notebook rerun
#MDCovid.head(5), SDOH.head(5), CDC.head(5), MDCensus.head(5), MDCrosswalk.head(5)

## Create Athena Tables from GitHub CSV and Excel Files

Coyne, S. (2022, October 9). 03_Register_S3_TSV_With_Athena.ipynb [Jupyter Notebook]. GitHub. https://github.com/MADS508/labs/blob/main/04_ingest/03_Register_S3_TSV_With_Athena.ipynb

### Create string variables to hold values for respective tables and the database for SQL statement

In [8]:
MDCovid_table = "MDCovid"
SDOH_table = "SDOH"
CDC_table = "CDC"
MDCensus_table = "MDCensus"
MDCrosswalk_table = "MDCrosswalk"
database_name = "covid"

### MD Covid by Zip Code Table

In [9]:
table_name = MDCovid_table
path = MDCovid_path

#TO DO: Figure out how to handle total{date} columns for schema. Columnar?
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         OBJECTID int,
         ZIP_CODE int,
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_name, path
)

print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS covid.MDCovid(
         OBJECTID int,
         ZIP_CODE int,
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' LOCATION 's3://ads508-team2-spring24/01_MD_COVID-19_-_Cases_by_ZIP_Code.csv'
TBLPROPERTIES ('skip.header.line.count'='1')


### Social Determinants of Health Database Table

In [10]:
table_name = SDOH_table
path = SDOH_path

#TO DO: Figure out how to handle the 327 columns. S3?
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         OBJECTID int,
         ZIP_CODE int,
)""".format(
    database_name, table_name, path
)

print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS covid.SDOH(
         OBJECTID int,
         ZIP_CODE int,
)


### PLACES: Local Data for Better Health Table

In [11]:
table_name = CDC_table
path = CDC_path

#TO DO: Figure out how to handle total{date} columns for schema. Columnar?
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         YEAR YEAR,
         LocationName int,
         DataSource string,
         Category string,
         Measure string,
         Data_Value_Unit string,
         Data_Value_Type string,
         Data_Value DECIMAL(3,1),
         Data_Value_Footnote_Symbol string,
         Data_Value_Footnote string,
         Low_Confidence_Limit DECIMAL(3,1),
         High_Confidence_Limit DECIMAL(3,1),
         TotalPopulation int,
         Geolocation string,
         LocationID int,
         CategoryID string,
         MeasureId string,
         DataValueTypeID string,
         Short_Question_Text string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_name, path
)

print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS covid.CDC(
         YEAR YEAR,
         LocationName int,
         DataSource string,
         Category string,
         Measure string,
         Data_Value_Unit string,
         Data_Value_Type string,
         Data_Value DECIMAL(3,1),
         Data_Value_Footnote_Symbol string,
         Data_Value_Footnote string,
         Low_Confidence_Limit DECIMAL(3,1),
         High_Confidence_Limit DECIMAL(3,1),
         TotalPopulation int,
         Geolocation string,
         LocationID int,
         CategoryID string,
         MeasureId string,
         DataValueTypeID string,
         Short_Question_Text string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' LOCATION 's3://ads508-team2-spring24/03_PLACES__Local_Data_for_Better_Health__ZCTA_Data_2023_release.csv'
TBLPROPERTIES ('skip.header.line.count'='1')


### Maryland Census Data Table

In [12]:
table_name = MDCensus_table
path = MDCensus_path

#TO DO: Figure out how to handle total{date} columns for schema. Columnar?
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         OBJECTID_1 int,
         ZCTA5CE10 int,
         FIRST_STAT int,
         FIRST_GEOI int,
         FIRST_CLAS CHAR(2),
         FIRST_MTFC string,
         FIRST FUNC CHAR(1),
         ZCTA5N int,
         STATE int,
         AREALAND int,
         AREAWATR int,
         POP100 int,
         HU100 int,
         NHW int,
         NHB int,
         NHAI int,
         NHA int,
         NHNH int,
         NHO int,
         NHT int,
         HISP int,
         PNHW DECIMAL(3,1),
         PNHB DECIMAL(3,1),
         PNHAI DECIMAL(2,1),
         PNHA DECIMAL(2,1),
         PNHNH DECIMAL(2,1),
         PNHT DECIMAL(2,1),
         PHISP DECIMAL(2,1),
         POP65_ int,
         PCTPOP65_ DECIMAL(3,1),
         MEDAGE DECIMAL(3,1),
         VACNS DECIMAL(2,1),
         PVACNS DECIMAL(2,1),
         PHOWN DECIMAL(3,1),
         PWOMORT DECIMAL(3,1),
         PRENT DECIMAL(3,1),
         PLT18SP DECIMAL(3,1),
         REPORT_2_P string,
         REPORT_9_P string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_name, path
)

print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS covid.MDCensus(
         OBJECTID_1 int,
         ZCTA5CE10 int,
         FIRST_STAT int,
         FIRST_GEOI int,
         FIRST_CLAS CHAR(2),
         FIRST_MTFC string,
         FIRST FUNC CHAR(1),
         ZCTA5N int,
         STATE int,
         AREALAND int,
         AREAWATR int,
         POP100 int,
         HU100 int,
         NHW int,
         NHB int,
         NHAI int,
         NHA int,
         NHNH int,
         NHO int,
         NHT int,
         HISP int,
         PNHW DECIMAL(3,1),
         PNHB DECIMAL(3,1),
         PNHAI DECIMAL(2,1),
         PNHA DECIMAL(2,1),
         PNHNH DECIMAL(2,1),
         PNHT DECIMAL(2,1),
         PHISP DECIMAL(2,1),
         POP65_ int,
         PCTPOP65_ DECIMAL(3,1),
         MEDAGE DECIMAL(3,1),
         VACNS DECIMAL(2,1),
         PVACNS DECIMAL(2,1),
         PHOWN DECIMAL(3,1),
         PWOMORT DECIMAL(3,1),
         PRENT DECIMAL(3,1),
         PLT18SP DECIMAL(3,1),
         REPORT_2_P string

### ZTCA to Zip Code Crosswalk Table
#### https://www.arcgis.com/home/item.html?id=dc123f738bf846779c49db6472f82a4b
#### https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.html

In [13]:
table_name = MDCrosswalk_table
path = MDCrosswalk_path

#TO DO: Figure out how to handle total{date} columns for schema. Columnar?
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         ZIP_CODE int,
         PO_NAME string,
         STATE string,
         ZIP_TYPE string,
         zcta int,
         zip_join_type string
)""".format(
    database_name, table_name, path
)

print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS covid.MDCrosswalk(
         ZIP_CODE int,
         PO_NAME string,
         STATE string,
         ZIP_TYPE string,
         zcta int,
         zip_join_type string
)


## Release Resources: Direct copy from

Coyne, S. (2024, February 27). Update 01_Copy_TSV_To_S3.ipynb [Jupyter Notebook]. GitHub. https://github.com/MADS508/labs/blob/main/04_ingest/01_Copy_TSV_To_S3.ipynb

In [14]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [15]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>